<a href="https://colab.research.google.com/github/kurakanja/113-2coding/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI 助理記帳系統（I/O to GoogleSheet + LLM + 條件判斷）

使用 Google Sheets 作為記帳工具
透過 LLM幫助分類消費類別（如餐飲、交通、娛樂），並匯入Google sheet與加總

試算表:
https://docs.google.com/spreadsheets/d/1BHVKroBrAX9O0Ygst-8kIlfZ1xKUqMb9UOUibw1ZqyE/edit?usp=sharing

此colab連結:https://colab.research.google.com/drive/1UjYIWQ7cdSMY58y78H2SQArCbBegoKxq?usp=sharing

In [ ]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1BHVKroBrAX9O0Ygst-8kIlfZ1xKUqMb9UOUibw1ZqyE/edit?gid=0#gid=0')

輸入數據

In [ ]:
 spending_records = []  # 用來存放所有消費紀錄
 while True:
        print("\n紀錄一筆消費 (輸入 '結束' 來結束)")
        spending_stuff = input("消費名稱: ")

        if spending_stuff == "結束":
            print("輸入結束，已存檔。")
            break  # 跳出迴圈

        spending_money = input("花了多少錢: ")

        # 檢查金額是否為數字
        if not spending_money.isdigit():
            print("請輸入有效的數字！")
            continue  # 重新輸入

        spending_money = int(spending_money)

        prompt = f'花費名稱: {spending_stuff} 請歸類此筆消費為哪種類別之消費(飲食、著裝、娛樂、生活、其他等，盡量不用分太細)並只回答類別名稱'

        response = model.generate_content(prompt)
        spending_type = response.text

        # 建立字典儲存這筆消費
        spending_record ={

            "名稱": spending_stuff,
            "金額": spending_money,
            "類別": spending_type
            }
        # 將這筆紀錄加入列表
        spending_records.append(spending_record)
        print("消費紀錄已儲存！")


紀錄一筆消費 (輸入 '結束' 來結束)
消費名稱: 啤酒
花了多少錢: 89
消費紀錄已儲存！

紀錄一筆消費 (輸入 '結束' 來結束)
消費名稱: 和朋友逛街
花了多少錢: 650
消費紀錄已儲存！

紀錄一筆消費 (輸入 '結束' 來結束)
消費名稱: 搭車
花了多少錢: 12
消費紀錄已儲存！

紀錄一筆消費 (輸入 '結束' 來結束)
消費名稱: 買襪子
花了多少錢: 65
消費紀錄已儲存！

紀錄一筆消費 (輸入 '結束' 來結束)
消費名稱: 吃雞排
花了多少錢: 99
消費紀錄已儲存！

紀錄一筆消費 (輸入 '結束' 來結束)
消費名稱: 亂花錢，忘了
花了多少錢: 165
消費紀錄已儲存！

紀錄一筆消費 (輸入 '結束' 來結束)
消費名稱: 結束
輸入結束，已存檔。


In [ ]:
total_spending = sum(record['金額'] for record in spending_records)
print(f" 本次總花費: {total_spending} 元")
spending_records

 本次總花費: 1080 元


[{'名稱': '啤酒', '金額': 89, '類別': '飲食\n'},
 {'名稱': '和朋友逛街', '金額': 650, '類別': '娛樂\n'},
 {'名稱': '搭車', '金額': 12, '類別': '生活\n'},
 {'名稱': '買襪子', '金額': 65, '類別': '著裝\n'},
 {'名稱': '吃雞排', '金額': 99, '類別': '飲食\n'},
 {'名稱': '亂花錢，忘了', '金額': 165, '類別': '其他\n'}]

In [ ]:
# 轉換成 DataFrame
from io import StringIO

# 轉換為 DataFrame

df = pd.DataFrame(spending_records)

# 去除「類別」欄位中的換行符號
df['類別'] = df['類別'].str.strip()

# 顯示 DataFrame
print(df)

       名稱   金額  類別
0      啤酒   89  飲食
1   和朋友逛街  650  娛樂
2      搭車   12  生活
3     買襪子   65  著裝
4     吃雞排   99  飲食
5  亂花錢，忘了  165  其他


In [ ]:

from gspread_dataframe import set_with_dataframe
# 選擇要更新的工作表（選擇第一個工作表）
worksheet_allcommand = gsheets.get_worksheet(0)  # 如果是其他工作表，可以用 get_worksheet(index)
# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet_allcommand, df)
# 在 Google Sheets 最後一行新增 "合計"
worksheet_allcommand.append_row(["合計", total_spending, ""])
print(f"✅ 已成功新增紀錄表格與合計至 Google Sheets！本次總花費: {total_spending} 元")

✅ 已成功新增紀錄表格與合計至 Google Sheets！本次總花費: 1080 元
